# MCP Genie Tool-Calling Agent with Databricks Claude Sonnet 4

This notebook demonstrates the **MCP Genie Agent** - a powerful tool-calling agent using Databricks' Claude Sonnet 4 endpoint with MCP (Model Context Protocol) Genie servers.

## What This Agent Does

🔍 **Query Databricks System Tables** - Ask natural language questions about your Databricks usage  
🤖 **Claude Sonnet 4 Powered** - Uses Databricks' most advanced LLM endpoint  
🔧 **Multi-MCP Server Support** - Extensible framework for multiple MCP servers  
🚀 **Production Ready** - Deploy to Databricks Playground via MLflow  

## Quick Start

1. **Development & Testing** - Use this notebook for interactive development
2. **Production Deployment** - Deploy to Databricks Playground for broader access

---

**💡 New in this version**: Clean Python package structure with deployment capabilities!

## Installation and Setup

First, let's install the required dependencies:

In [1]:
# Install all required packages from requirements.txt
%pip install -r requirements.txt

print("✅ All required packages installed from requirements.txt!")
print("📋 Next: Set up OAuth authentication using the .env file")


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ All required packages installed from requirements.txt!
📋 Next: Set up OAuth authentication using the .env file


## Import Required Libraries

Import all necessary libraries for the MCP tool-calling agent:

In [ ]:
import asyncio
import os

# Import the MCP agent package
from src.agent import MCPAgent, SingleTurnMCPAgent
from src.mcp_client import MCPServerManager, GenieServerClient
from config import config, validate_oauth_setup

print("✅ MCP Agent package imported successfully!")
print(f"🔗 LLM Endpoint: {config.llm_endpoint_name}")
print(f"🌐 Workspace: {config.databricks_host}")
print(f"🗂️ Genie Space ID: {config.genie_space_id}")

## Configuration

Configure the LLM endpoint and system prompt. We'll use Databricks' **Claude Sonnet 4** endpoint:

In [ ]:
# Configuration loaded from config.py
# LLM and system prompt configuration - defaults to Claude Sonnet 4
llm = ChatDatabricks(endpoint=config.llm_endpoint_name)
system_prompt = config.system_prompt

print("✅ Configuration loaded from config.py")
print(f"🔗 LLM Endpoint: {config.llm_endpoint_name} (Claude Sonnet 4)")
print(f"🌐 Workspace: {config.databricks_host}")
print(f"🗂️ Genie Space ID: {config.genie_space_id}")
print(f"📡 MCP Server URL: {config.genie_server_url}")

## Agent State Definition

Define the state structure for our LangGraph agent:

In [ ]:
# Configure MCP tools and agent workflow
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    custom_inputs: Optional[dict[str, Any]]

## MCP Genie Client Setup

Set up the MCP client to connect to the Genie server. **Note**: You'll need to provide the Genie space ID and authentication details.

In [ ]:
# Configuration is now managed in config.py
# This provides better security and organization

print("✅ Configuration externalized to config.py")
print("📁 All settings are now managed in the config file")
print("🔒 OAuth credentials are loaded from environment variables")
print("\n💡 To configure your credentials, choose one of these methods:")
print()
print("Method 1: Environment Variables")
print('export DATABRICKS_HOST="https://your-workspace.cloud.databricks.com"')
print('export DATABRICKS_CLIENT_ID="your-service-principal-client-id"')
print('export DATABRICKS_CLIENT_SECRET="your-oauth-secret"')
print('export GENIE_SPACE_ID="your-genie-space-id"')
print()
print("Method 2: Programmatic Setup (see next cell)")
print()
print("Method 3: .databrickscfg file")
print("[default]")
print("host = https://your-workspace.cloud.databricks.com")
print("client_id = your-service-principal-client-id")
print("client_secret = your-oauth-secret")

## OAuth Authentication Setup

**This notebook requires OAuth authentication with Service Principal credentials.**

### Step 1: Create Service Principal (One-time setup)

1. **In your Databricks workspace**:
   - Go to **Settings** → **Identity and access** → **Service principals**
   - Click **Add service principal**
   - Enter name: `MCP-Genie-Agent`
   - Click **Add**

2. **Generate OAuth credentials**:
   - Select your service principal
   - Go to **OAuth secrets** tab
   - Click **Generate secret**
   - Set lifetime (max 730 days, recommended: 365 days)
   - **⚠️ Copy the Client ID and Client Secret** (shown only once!)

3. **Assign workspace permissions**:
   - Add the service principal to your workspace
   - Grant necessary permissions for Genie space access

### Step 2: Configure OAuth Authentication

Choose one of the following authentication methods:

In [ ]:
# Programmatic Configuration Setup
# Use this if you prefer to set credentials directly in the notebook

# Uncomment and configure these with your actual values:
# config.set_oauth_credentials(
#     client_id="your-service-principal-client-id",
#     client_secret="your-oauth-secret", 
#     workspace_host="your-workspace.cloud.databricks.com"
# )
# config.set_genie_space_id("your-genie-space-id")

print("🔧 Programmatic configuration helper available")
print("📝 Uncomment and modify the config.set_oauth_credentials() call above")
print("⚠️  Remember: Don't commit actual credentials to version control!")

## OAuth Authentication Test

Run this cell to verify your OAuth setup is working correctly:

In [ ]:
# Configuration Validation
# This uses the config.py validation functions

def test_oauth_configuration():
    """Test OAuth authentication configuration using config.py."""
    print("🧪 Testing OAuth Configuration...")
    print("=" * 50)
    
    # Check for conflicting authentication methods
    import os
    if "DATABRICKS_TOKEN" in os.environ:
        print("⚠️  Warning: DATABRICKS_TOKEN is set, which conflicts with OAuth!")
        print("   Unsetting DATABRICKS_TOKEN for this session...")
        del os.environ["DATABRICKS_TOKEN"]
    
    # Use the validation function from config.py
    is_valid = validate_oauth_setup()
    
    if not is_valid:
        return False
    
    # Test Databricks WorkspaceClient initialization
    try:
        print("\n🔄 Testing Databricks WorkspaceClient...")
        workspace_client = WorkspaceClient()
        
        # Test authentication by getting current user
        current_user = workspace_client.current_user.me()
        print(f"✅ Authentication successful!")
        print(f"   User: {current_user.user_name}")
        print(f"   Active: {current_user.active}")
        
        return True
        
    except Exception as e:
        print(f"❌ OAuth authentication failed: {e}")
        print("\n🔧 Troubleshooting steps:")
        print("1. Verify Client ID and Client Secret are correct")
        print("2. Check that the service principal is assigned to the workspace")
        print("3. Ensure the OAuth secret hasn't expired")
        print("4. Verify the workspace hostname is correct")
        return False

# Run the test
oauth_success = test_oauth_configuration()

if oauth_success:
    print(f"\n🎉 OAuth configuration is ready!")
    print(f"🚀 You can now proceed to test the MCP Genie agent")
else:
    print(f"\n⚠️  Please fix the OAuth configuration before proceeding")

## Development Agent Setup

Create and initialize the development agent for interactive use:

In [ ]:
async def create_development_agent():
    """Create and initialize the development MCP agent."""
    print("🔄 Creating development agent...")
    
    # Check for OAuth token conflicts
    if "DATABRICKS_TOKEN" in os.environ:
        print("⚠️  Removing DATABRICKS_TOKEN to avoid OAuth conflicts...")
        del os.environ["DATABRICKS_TOKEN"]
    
    # Create server manager
    server_manager = MCPServerManager()
    
    # Add Genie server
    from databricks.sdk import WorkspaceClient
    workspace_client = WorkspaceClient()
    genie_client = GenieServerClient(config.genie_server_url, workspace_client)
    server_manager.add_server("genie", genie_client)
    
    # Create the agent
    agent = MCPAgent(
        llm_endpoint=config.llm_endpoint_name,
        system_prompt=config.system_prompt,
        server_manager=server_manager
    )
    
    # Initialize the agent
    await agent.initialize()
    
    print("✅ Development agent ready!")
    return agent

# Create the agent
dev_agent = await create_development_agent()

## Test the Agent

Try asking questions about your Databricks usage:

In [ ]:
# Test with a simple query
query = "How many queries were executed over the past 7 days in SQL?"
response = await dev_agent.query(query)
print(f"🔍 Query: {query}")
print(f"📝 Response: {response}")

## Interactive Testing

Ask your own questions about Databricks usage:

In [ ]:
# Try different queries - modify this cell to test various questions
test_queries = [
    "What are the most expensive clusters by compute cost?",
    "Show me the top SQL queries by execution time",
    "Which users are most active in this workspace?",
    "What is the total data processed this month?"
]

# Test multiple queries
for query in test_queries[:2]:  # Test first 2 queries
    print(f"\n🔍 Query: {query}")
    try:
        response = await dev_agent.query(query)
        print(f"📝 Response: {response[:300]}...")  # Show first 300 chars
    except Exception as e:
        print(f"❌ Error: {e}")

## 🚀 Deploy to Databricks Playground

Ready to deploy your agent for broader access? Run the deployment script:

In [ ]:
# Deploy the agent to Databricks
# This will make it available in the Databricks Playground

# Option 1: Deploy from this notebook
from deployment.deploy_agent import main as deploy_main

print("🚀 Starting deployment to Databricks...")
print("This will:")
print("  ✅ Create MLflow model")
print("  ✅ Register in Unity Catalog") 
print("  ✅ Deploy serving endpoint")
print("  ✅ Make available in Playground")
print()

# Uncomment the line below to deploy:
# deploy_main()

# Option 2: Deploy from command line
print("🖥️  Alternative: Deploy from terminal")
print()
print("Run this command in your terminal:")
print("  python deployment/deploy_agent.py")
print()
print("📋 Deployment will:")
print("  • Validate configuration")  
print("  • Create MLflow experiment")
print("  • Log and register the model") 
print("  • Deploy serving endpoint")
print("  • Make agent available in Databricks Playground")
print()
print("🎮 After deployment, find your agent in:")
print(f"   Databricks Playground → Select Model → MCP Genie Agent")

In [ ]:
## 🎉 Next Steps

### **Development Complete!**
- ✅ **Clean Architecture** - Modular Python package structure
- ✅ **Multi-MCP Support** - Extensible for additional MCP servers  
- ✅ **Production Ready** - MLflow deployment capabilities
- ✅ **Playground Access** - Deploy for organization-wide use

### **What You Built:**
- 🔍 **Natural Language Queries** - Ask questions about Databricks usage in plain English
- 📊 **System Table Access** - Query compute, storage, and usage metrics
- 🤖 **Claude Sonnet 4** - Powered by Databricks' most advanced LLM
- 🚀 **Scalable Deployment** - Ready for production use

### **Try These Queries:**
- "How many queries were executed over the past 7 days in SQL?"
- "What are the most expensive clusters by compute cost?"
- "Show me the top SQL queries by execution time"
- "Which users are most active in this workspace?"

Your MCP Genie Agent is ready to help analyze Databricks data! 🎊

## Testing the Agent

Test the agent with some sample queries (will work once authentication is configured):

In [ ]:
async def test_agent():
    """
    Test the MCP Genie agent with sample queries.
    """
    print("Creating MCP Genie Agent...")
    
    try:
        # Create the agent
        agent_graph = await create_agent()
        agent = MCPGenieAgent(agent_graph)
        
        print("✅ Agent created successfully!")
        
        # Test queries
        test_queries = [
            "What data sources are available in this Genie space?",
            "Can you show me a summary of the available tables?",
            "What insights can you provide about the data?"
        ]
        
        for query in test_queries:
            print(f"\n🔍 Testing query: {query}")
            
            from langchain_core.messages import HumanMessage
            state = AgentState(
                messages=[HumanMessage(content=query)],
                custom_inputs=None
            )
            
            try:
                # Direct agent graph invocation for testing
                result = agent_graph.invoke(state)
                response = result["messages"][-1].content
                print(f"📝 Response: {response}")
            except Exception as e:
                print(f"❌ Error processing query: {e}")
        
        return agent
        
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        print("Please ensure OAuth authentication is properly configured.")
        return None

# Jupyter-friendly async execution
import asyncio
import nest_asyncio
nest_asyncio.apply()

print("✅ Test function defined. Run the cell below to test the agent.")
print("🔐 Make sure OAuth authentication is configured in the .env file!")

## Interactive Usage Example

Example of how to use the agent interactively:

In [ ]:
async def interactive_session():
    """
    Run an interactive session with the MCP Genie agent.
    """
    print("Starting interactive session with MCP Genie Agent...")
    print("Type 'quit' to exit\n")
    
    # Create agent
    agent_graph = await create_agent()
    
    # Initialize conversation state
    conversation_state = AgentState(messages=[], custom_inputs=None)
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        
        # Add user message to conversation
        from langchain_core.messages import HumanMessage
        user_message = HumanMessage(content=user_input)
        conversation_state["messages"].append(user_message)
        
        try:
            # Get agent response
            result = agent_graph.invoke(conversation_state)
            
            # Update conversation state
            conversation_state = result
            
            # Display response
            response = result["messages"][-1].content
            print(f"Agent: {response}\n")
            
        except Exception as e:
            print(f"Error: {e}\n")

# Run interactive session once OAuth is configured
print("✅ Interactive session function defined. Run 'await interactive_session()' once OAuth authentication is configured.")
print("🔐 OAuth authentication must be configured first!")

## Quick Start Guide

Once you have OAuth authentication configured, run these cells to test the agent:

In [ ]:
# Run this cell to test the agent with sample queries
agent = await test_agent()

In [ ]:
# Interactive Chat - Run this for a conversational interface
# Type your questions and the agent will respond using Genie data
# Type 'quit' to exit

await interactive_session()

In [ ]:
# Single Query Test - Run this to test with a custom query
# Modify the query below to test specific questions

async def single_query_test(query: str):
    """Test a single query with the agent."""
    print(f"🔍 Testing query: {query}")
    
    try:
        # Create agent if not already created
        agent_graph = await create_agent()
        
        # Create state with the query
        from langchain_core.messages import HumanMessage
        state = AgentState(
            messages=[HumanMessage(content=query)],
            custom_inputs=None
        )
        
        # Get response
        result = agent_graph.invoke(state)
        response = result["messages"][-1].content
        
        print(f"📝 Response: {response}")
        return response
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Example usage - modify the query as needed
query = "How many queries were executed over the past 7 days in SQL?"
response = await single_query_test(query)

## 🎉 Ready to Use!

The MCP Genie Agent is now configured and ready to query your Databricks system tables through natural language.

### Example Queries to Try:
- "How many queries were executed over the past 7 days in SQL?"
- "What are the most expensive clusters by compute cost?" 
- "Show me the top SQL queries by execution time"
- "Which users are running the most jobs?"
- "What is the total data processed in the last month?"

The agent will convert these natural language questions into SQL queries against your Databricks system tables and return the results! 🚀